In [1]:
import pandas as pd 
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df2 = pd.read_pickle('/content/drive/MyDrive/Grupo Facens DataScience/TCC/Grupo 1/Codes/Arquivos/INFLUD21-27-09-2021-TratadoWDummies_Inv1_NaoInv2_Nao2.pkl')

In [ ]:
y = df2['SUPORT_VEN'] 
x = df2.drop(['SUPORT_VEN'],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
def model_run(x,y):
    model = {"Bernoulli" : BernoulliNB(), 'ComplementNB' : ComplementNB(), 'MultinomialNB':  MultinomialNB() } 
    
    acuracy = {}
    for M in model.keys():
    
        Model = model[M]
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        acuracy[M] = acc 
    
    return acuracy

def MultiConfusionMatrix(y_test, y_pred):
    labels = list(y_test.unique())
    cf = multilabel_confusion_matrix(y_test, y_pred, labels=labels) 
    matrix = {'Classe' : [], 
             'VP':[],
             'FP':[],
             'VN':[],
             'FN':[]}
    for x, Class in enumerate(labels): 
        VN = cf[x][0][0]
        FN = cf[x][1][0]
        VP = cf[x][1][1]
        FP = cf[x][0][1]
        matrix['Classe'].append(Class)
        matrix['VP'].append(VP)
        matrix['FP'].append(FP)
        matrix['VN'].append(VN)
        matrix['FN'].append(FN)
    return pd.DataFrame(matrix)

In [ ]:
analise = {'Bernoulli': [],
            'ComplementNB': [],
            'MultinomialNB': []}

#Opção 1 - URL
acc = model_run(X_train, y_train)

analise['Bernoulli'].append(acc["Bernoulli"])
analise['ComplementNB'].append(acc['ComplementNB'])
analise['MultinomialNB'].append(acc['MultinomialNB'])

analise

{'Bernoulli': [0.7275588514270932],
 'ComplementNB': [0.5863046477537294],
 'MultinomialNB': [0.7628050357851168]}

In [ ]:
# Rodar Gridsearch com decisiontree
grid_params = {
    'criterion' : ['gini','entropy'],
    'splitter' :  ['best', 'random'],
    'max_depth' : [3,4,7,8,9],
    'max_features' : ['auto', 'sqrt', 'log2']}

gs = GridSearchCV(DecisionTreeClassifier(), grid_params, verbose = 1, cv = 3, n_jobs = -1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

Fitting 3 folds for each of 60 candidates, totalling 180 fits


{'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'sqrt',
 'splitter': 'best'}

In [ ]:
# Rodar Gridsearch com Logistic Regression Classifier
grid_params = { 
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

gs = GridSearchCV(LogisticRegression(), grid_params, verbose = 1, cv = 3, n_jobs = -1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

Fitting 3 folds for each of 15 candidates, totalling 45 fits


C:\Users\darci\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
3 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\darci\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\darci\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py", line 1519, in fit
    multi_class = _check_multi_class(self.multi_class, solver, len(self.classes_))
  File "C:\Users\darci\AppData\Roaming\Python\Python38\site-packages\s

{'multi_class': 'multinomial', 'solver': 'newton-cg'}

In [ ]:
#Rodando Gridsearch p/ KNN
grid_params = {
    'n_neighbors' : [3,5,7],
    'weights' : ['uniform','distance'],
    'metric' : ['chebyshev','manhattan','euclidean'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv = 2, n_jobs = 1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

Fitting 2 folds for each of 72 candidates, totalling 144 fits


{'algorithm': 'auto',
 'metric': 'chebyshev',
 'n_neighbors': 7,
 'weights': 'uniform'}

In [ ]:
model = MultinomialNB()
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de MultinomialNB:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('-'*80)
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print(' VN',vn,'\n','FP', fp,'\n', 'FN', fn,'\n', 'VP',vp,'\n')
print('-'*80)
print(classification_report(y_test, y_pred))
print('-'*80)
print('\n')

model = KNeighborsClassifier(algorithm='auto', metric='chebyshev', n_neighbors= 7, weights='uniform')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Knn:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('-'*80)
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print(' VN',vn,'\n','FP', fp,'\n', 'FN', fn,'\n', 'VP',vp,'\n')
print('-'*80)
print(classification_report(y_test, y_pred))
print('-'*80)
print('\n')

model = DecisionTreeClassifier(criterion='gini', max_depth = 7, max_features='sqrt', splitter='best')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Decision Tree:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('-'*80)
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print(' VN',vn,'\n','FP', fp,'\n', 'FN', fn,'\n', 'VP',vp,'\n')
print('-'*80)
print(classification_report(y_test, y_pred))
print('-'*80)
print('\n')

model = RandomForestClassifier(max_features='log2', n_estimators= 700)
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Random Forest:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('-'*80)
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print(' VN',vn,'\n','FP', fp,'\n', 'FN', fn,'\n', 'VP',vp,'\n')
print('-'*80)
print(classification_report(y_test, y_pred))
print('-'*80)
print('\n')

model = LogisticRegression(multi_class='multinomial', solver='newton-cg')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Logistic Regression:', round(scores.mean(),4),' - ', round(scores.std(),2))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('-'*80)
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print(' VN',vn,'\n','FP', fp,'\n', 'FN', fn,'\n', 'VP',vp,'\n')
print('-'*80)
print(classification_report(y_test, y_pred))
print('-'*80)
print('\n')


Acurácia de MultinomialNB: 0.7627934965458559
--------------------------------------------------------------------------------
 VN 1464 
 FP 19709 
 FN 2297 
 VP 69306 

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           1       0.39      0.07      0.12     21173
           2       0.78      0.97      0.86     71603

    accuracy                           0.76     92776
   macro avg       0.58      0.52      0.49     92776
weighted avg       0.69      0.76      0.69     92776

--------------------------------------------------------------------------------


Acurácia de Knn: 0.761751306349997
--------------------------------------------------------------------------------
 VN 1445 
 FP 19728 
 FN 2753 
 VP 68850 

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           1       0.34      0.07      0.11 